In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [3]:
# 读取数据集
iris_data = pd.read_csv('D:\\Users\\my_projects\\vscode_projects\\deep_learning\\datasets/iris.data')
display(iris_data.head())
# adm_data = pd.read_csv('../adm_data.csv')
# display(adm_data.head())

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
# 数据处理：将样本标签进行编码，并将数据集分为训练集和测试集
le = LabelEncoder()
le.fit(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])
iris_data['class'] = le.transform(iris_data['class'])

features = iris_data.columns.to_list()
features.remove('class')

train_x, test_x, train_y, test_y = train_test_split(iris_data[features], iris_data['class'],
                                                    test_size=0.2, shuffle=True, random_state=2023)

## 回归数据集

In [4]:
features_2 = adm_data.columns.to_list()

train_x2, test_x2, train_y2, test_y2 = train_test_split(adm_data[features_2[1: -1]], adm_data[features_2[-1]],
                                                    test_size=0.2, shuffle=True, random_state=2023)

train_x2_np, test_x2_np = np.array(train_x2, dtype=np.float32), np.array(test_x2, dtype=np.float32)
train_y2_np, test_y2_np = np.array(train_y2, dtype=np.float32), np.array(test_y2, dtype=np.float32)

train_x2_t, test_x2_t = torch.from_numpy(train_x2_np), torch.from_numpy(test_x2_np)
train_y2_t, test_y2_t = torch.from_numpy(train_y2_np), torch.from_numpy(test_y2_np)

trainset2 = TensorDataset(train_x2_t, train_y2_t)
testset2 = TensorDataset(test_x2_t, test_y2_t)

trainloader2 = DataLoader(trainset2, batch_size=2, shuffle=False)
testloader2 = DataLoader(testset2, batch_size=1, shuffle=False)

## 分类数据集

In [5]:
train_x_np, test_x_np, train_y_np, test_y_np = np.array(train_x, dtype=np.float32), np.array(test_x, dtype=np.float32), np.array(train_y, dtype=np.float32), np.array(test_y, dtype=np.float32)

train_x_t, test_x_t = torch.from_numpy(train_x_np), torch.from_numpy(test_x_np)
train_y_t, test_y_t = torch.from_numpy(train_y_np), torch.from_numpy(test_y_np)

trainset = TensorDataset(train_x_t, train_y_t)
testset = TensorDataset(test_x_t, test_y_t)

trainloader = DataLoader(trainset, batch_size=1, shuffle=False)
testloader = DataLoader(testset, batch_size=1, shuffle=False)

In [6]:
def one_hot(x):
    y = np.zeros(3)
    y[int(x)] = 1
    return y

## pytorch

In [7]:
class torchBP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.fc1 = nn.Linear(input_dims, hidden_dims)
        self.fc2 = nn.Linear(hidden_dims, output_dims)


    def forward(self, x):
        output = nn.functional.relu(self.fc1(x))
        output = self.fc2(output)
        # output = nn.functional.softmax(output)
        return output

## numpy

In [14]:
class npBP():
    def __init__(self, input_dims, hidden_dims, output_dims) -> None:
        self.ids = input_dims
        self.hds = hidden_dims
        self.ods = output_dims

        self.w1 = np.random.randn(self.ids, self.hds) * 0.001
        self.b1 = np.zeros(self.hds)
        self.w2 = np.random.randn(self.hds, self.ods) * 0.001
        self.b2 = np.zeros(self.ods)


    def f(self, x):
        # RELU
        return np.where(x > 0, x, 0)
    
    def df(self, x):
        return np.where(x > 0, 1, 0)


    def forward(self, x):
        self.z1 = np.dot(x, self.w1) + self.b1
        self.a1 = self.f(self.z1)
        self.a1 = self.a1.reshape(1, -1)
        self.z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = self.z2

        return self.a2
    

    def back_propagation(self, x, y, y_hat, lr):
        dE_da2 = y_hat - y
        dE_dz2 = dE_da2 * self.df(self.z2)
        dE_dw2 = dE_dz2 * self.a1.T
        dE_db2 = dE_dz2.reshape(-1)
        
        dE_dz1 = np.dot(self.w2, dE_dz2.T).T * self.df(self.z1)
        dE_dw1 = np.dot(x.reshape(-1, 1), dE_dz1)
        dE_db1 = dE_dz1.reshape(-1)


        self.w1 -= lr * dE_dw1
        self.b1 -= lr * dE_db1
        self.w2 -= lr * dE_dw2
        self.b2 -= lr * dE_db2


    def train(self, X, y, epochs, lr):
        for epoch in range(epochs):
            train_loss = 0.
            for x, target in zip(X, y):
                y_hat = self.forward(x)
                y_hat = y_hat.item()
                self.back_propagation(x, target, y_hat, lr)
                train_loss += y_hat - target
            
            if (epoch + 1) % 50 == 0:
                train_loss = np.sum(np.abs(train_loss)) / len(X)
                print(f'Epoch {epoch + 1}\ttrain loss: {train_loss:.4f}')


    def predict(self, X):
        pred = self.forward(X)
        return pred

## pytorch 回归

In [11]:
ttbp = torchBP(7, 128, 1)
lr, epochs = 0.00001, 250
optimizer = torch.optim.Adam(ttbp.parameters(), lr=lr)
loss_func = nn.MSELoss()


def train():
    for epoch in range(epochs):
        train_loss = 0.
        for x, y in trainloader2:
            optimizer.zero_grad()
            output = ttbp(x)
            loss = loss_func(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        if (epoch + 1) % 50 == 0:
            train_loss /= len(trainloader2.dataset)
            print(f'Epoch {epoch + 1}\ttrain loss: {train_loss:.4f}')


def test():
    c = 0
    with torch.no_grad():
        for x, y in testloader2:
            output = ttbp(x)

            if c <= 9:
                print(f'sample {c}\nlabel: {y}\tthe prediction is {output}\n')
                c += 1

In [12]:
train()
test()

Epoch 50	train loss: 0.0096
Epoch 100	train loss: 0.0091
Epoch 150	train loss: 0.0091
Epoch 200	train loss: 0.0090
Epoch 250	train loss: 0.0090
sample 0
label: tensor([0.9300])	the prediction is tensor([[0.7694]])

sample 1
label: tensor([0.7200])	the prediction is tensor([[0.7365]])

sample 2
label: tensor([0.5700])	the prediction is tensor([[0.6722]])

sample 3
label: tensor([0.6200])	the prediction is tensor([[0.7370]])

sample 4
label: tensor([0.6700])	the prediction is tensor([[0.6351]])

sample 5
label: tensor([0.6400])	the prediction is tensor([[0.6800]])

sample 6
label: tensor([0.7100])	the prediction is tensor([[0.6560]])

sample 7
label: tensor([0.7100])	the prediction is tensor([[0.7119]])

sample 8
label: tensor([0.5400])	the prediction is tensor([[0.7065]])

sample 9
label: tensor([0.6900])	the prediction is tensor([[0.6816]])



## numpy 回归

In [15]:
nbp = npBP(7, 128, 1)
nbp.train(train_x2_np, train_y2_np, 250, 0.001)
c = 0
for x, y in zip(test_x2_np, test_y2_np):
    pred = nbp.predict(x)
    # 查看 10 个样本的具体输出
    if c <= 9:
        print(f'sample {c}\nlabel: {y:.4f}\tthe prediction is {pred[0]}\n')
        c += 1

Epoch 50	train loss: 0.0009
Epoch 100	train loss: 0.0009
Epoch 150	train loss: 0.0008
Epoch 200	train loss: 0.0007
Epoch 250	train loss: 0.0006
sample 0
label: 0.9300	the prediction is [0.79307572]

sample 1
label: 0.7200	the prediction is [0.71767747]

sample 2
label: 0.5700	the prediction is [0.71138987]

sample 3
label: 0.6200	the prediction is [0.73888324]

sample 4
label: 0.6700	the prediction is [0.63814836]

sample 5
label: 0.6400	the prediction is [0.67890214]

sample 6
label: 0.7100	the prediction is [0.68573848]

sample 7
label: 0.7100	the prediction is [0.67258561]

sample 8
label: 0.5400	the prediction is [0.67625791]

sample 9
label: 0.6900	the prediction is [0.7092448]



## numpy 分类

In [24]:
nbp = npBP(4, 128, 1)
nbp.train(train_x_np, train_y_np, 100, 0.01)

correct = 0
total = 0
c = 0
for x, y in zip(test_x_np, test_y_np):
    pred = np.round(nbp.predict(x))
    if pred == y:
        correct += 1
    total += 1
    # 查看 10 个样本的具体输出
    if c <= 9:
        print(f'sample {c}\nfeatures: {x}\tlabel: {y}\tthe prediction is {pred}\n')
        c += 1

print(f'the accuracy: {correct / total * 100}%')

sample 0
features: [6.4 2.8 5.6 2.1]	label: 2.0	the prediction is [[2.]]

sample 1
features: [6.4 2.9 4.3 1.3]	label: 1.0	the prediction is [[1.]]

sample 2
features: [5.7 2.8 4.1 1.3]	label: 1.0	the prediction is [[1.]]

sample 3
features: [7.3 2.9 6.3 1.8]	label: 2.0	the prediction is [[2.]]

sample 4
features: [6.8 2.8 4.8 1.4]	label: 1.0	the prediction is [[1.]]

sample 5
features: [5.7 2.5 5.  2. ]	label: 2.0	the prediction is [[2.]]

sample 6
features: [5.4 3.  4.5 1.5]	label: 1.0	the prediction is [[1.]]

sample 7
features: [5.7 2.9 4.2 1.3]	label: 1.0	the prediction is [[1.]]

sample 8
features: [5.  3.6 1.4 0.2]	label: 0.0	the prediction is [[0.]]

sample 9
features: [6.  2.2 4.  1. ]	label: 1.0	the prediction is [[1.]]

the accuracy: 100.0%


## pytorch 分类

In [19]:
lr, epochs = 0.01, 100
tbp = torchBP(4, 32, 3)
optimizer = torch.optim.SGD(tbp.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()


def train():
    for epoch in range(epochs):
        train_loss = 0.
        for x, y in trainloader:
            optimizer.zero_grad()
            output = tbp(x)
            y = y.long()
            loss = loss_func(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        if (epoch + 1) % 10 == 0:
            train_loss /= len(trainloader.dataset)
            print(f'Epoch {epoch + 1}\ttrain loss: {train_loss:.4f}')
        

def test():
    correct = 0
    total = 0
    c = 0
    with torch.no_grad():
        for x, y in testloader:
            output = tbp(x)
            output = torch.argmax(output)
            correct += (output == y).sum().item()
            total += len(x)
            if c <= 9:
                print(f'sample {c}\nfeatures: {x}\tlabel: {y}\tthe prediction is {output}\n')
                c += 1
        
    print(f'the accuracy: {correct / total * 100}%')

In [20]:
train()
test()

Epoch 10	train loss: 0.2661
Epoch 20	train loss: 0.2017
Epoch 30	train loss: 0.1778
Epoch 40	train loss: 0.1648
Epoch 50	train loss: 0.1568
Epoch 60	train loss: 0.1509
Epoch 70	train loss: 0.1462
Epoch 80	train loss: 0.1420
Epoch 90	train loss: 0.1385
Epoch 100	train loss: 0.1353
sample 0
features: tensor([[6.4000, 2.8000, 5.6000, 2.1000]])	label: tensor([2.])	the prediction is 2

sample 1
features: tensor([[6.4000, 2.9000, 4.3000, 1.3000]])	label: tensor([1.])	the prediction is 1

sample 2
features: tensor([[5.7000, 2.8000, 4.1000, 1.3000]])	label: tensor([1.])	the prediction is 1

sample 3
features: tensor([[7.3000, 2.9000, 6.3000, 1.8000]])	label: tensor([2.])	the prediction is 2

sample 4
features: tensor([[6.8000, 2.8000, 4.8000, 1.4000]])	label: tensor([1.])	the prediction is 1

sample 5
features: tensor([[5.7000, 2.5000, 5.0000, 2.0000]])	label: tensor([2.])	the prediction is 2

sample 6
features: tensor([[5.4000, 3.0000, 4.5000, 1.5000]])	label: tensor([1.])	the prediction is 1